In [1]:
from utils import NerDataset

dev_dataset = NerDataset('./processed/processed_dev_bio.txt')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 109540/109540 [00:00<00:00, 138765.53B/s]


In [2]:
len(dev_dataset.sents)

100

In [3]:
train_dataset = NerDataset('./processed/processed_training_bio.txt')
len(train_dataset)

900

In [4]:
test_dataset = NerDataset('./processed/processed_test_bio.txt')
len(test_dataset)

1000

In [5]:
test_dataset[0]

('[CLS] ， 患 者 3 月 前 因 “ 直 肠 癌 ” 于 在 我 院 于 全 麻 上 行 直 肠 癌 根 治 术 （ D I X O N 术 ） ， 手 术 过 程 顺 利 ， 术 后 给 予 抗 感 染 及 营 养 支 持 治 疗 ， 患 者 恢 复 好 ， 切 口 愈 合 良 好 。 ， 术 后 病 理 示 ： 直 肠 腺 癌 （ 中 低 度 分 化 ） ， 浸 润 溃 疡 型 ， 面 积 3 . 5 * 2 C M ， 侵 达 外 膜 。 双 端 切 线 另 送 “ 近 端 ” 、 “ 远 端 ” 及 环 周 底 部 切 除 面 未 查 见 癌 。 肠 壁 一 站 （ 1 0 个 ） 、 中 间 组 （ 8 个 ） 淋 巴 结 未 查 见 癌 。 ， 免 疫 组 化 染 色 示 ： E R C C 1 弥 漫 （ + ） 、 T S 少 部 分 弱 （ + ） 、 S Y N （ - ） 、 C G A （ - ） 。 术 后 查 无 化 疗 禁 忌 后 给 予 3 周 期 化 疗 ， ， 方 案 为 ： 奥 沙 利 铂 1 5 0 M G O D 1 ， 亚 叶 酸 钙 0 . 3 G + 替 加 氟',
 [101,
  8024,
  2642,
  5442,
  124,
  3299,
  1184,
  1728,
  100,
  4684,
  5499,
  4617,
  100,
  754,
  1762,
  2769,
  7368,
  754,
  1059,
  7937,
  677,
  6121,
  4684,
  5499,
  4617,
  3418,
  3780,
  3318,
  8020,
  146,
  151,
  166,
  157,
  156,
  3318,
  8021,
  8024,
  2797,
  3318,
  6814,
  4923,
  7556,
  1164,
  8024,
  3318,
  1400,
  5314,
  750,
  2834,
  2697,
  3381,
  1350,
  5852,
  1075,
  3118,
  2898,
  3780,
  4545,
  8024,
  2642,
  5442,
  261

In [9]:
train_dataset[1]

('[CLS] ， 患 者 因 罹 患 “ 胃 癌 ” 于 2 0 1 3 - 1 0 - 2 9 在 我 院 予 行 全 麻 上 胃 癌 根 治 术 ， ， 术 中 见 ： 腹 腔 内 腹 水 ， 腹 膜 无 转 移 ， 肝 脏 未 触 及 明 显 转 移 性 灶 ， 肿 瘤 位 于 胃 体 、 胃 底 部 ， 小 弯 侧 偏 后 壁 ， 约 5 * 4 * 2 C M 大 小 ， 肿 瘤 已 侵 达 浆 膜 外 ， 第 1 、 3 组 淋 巴 结 肿 大 ， 肿 瘤 尚 能 活 动 ， 经 探 查 决 定 行 全 胃 切 除 ， 空 肠 J 字 代 胃 术 。 手 术 顺 利 ， 术 后 积 极 予 相 关 对 症 支 持 治 疗 ； ， 后 病 理 示 ： 胃 底 、 体 小 弯 侧 低 分 化 腺 癌 ， 部 分 为 印 戒 细 胞 癌 图 像 ， 蕈 伞 型 ， 面 积 5 . 2 * 3 . 5 C M ， 局 部 侵 达 粘 膜 上 层 ， 并 于 少 数 腺 管 内 查 见 癌 栓 。 双 端 切 线 及 另 送 “ 近 端 切 线 ” 未 查 见 癌 。 呈 三 组 （ 5 / 1 3 个 ） 淋 巴',
 [101,
  8024,
  2642,
  5442,
  1728,
  5395,
  2642,
  100,
  5517,
  4617,
  100,
  754,
  123,
  121,
  122,
  124,
  118,
  122,
  121,
  118,
  123,
  130,
  1762,
  2769,
  7368,
  750,
  6121,
  1059,
  7937,
  677,
  5517,
  4617,
  3418,
  3780,
  3318,
  8024,
  8024,
  3318,
  704,
  6224,
  8038,
  5592,
  5579,
  1079,
  5592,
  3717,
  8024,
  5592,
  5606,
  3187,
  6760,
  4919,
  8024,
  5498,
  5552,
  3313,
  6239,
  1350,
  3209,
  3227,
  6760,
  4919,
 

### 分段处理，使长度低于256

In [10]:
with open('./processed/processed_training_bio.txt', 'r', encoding='utf-8') as f:
    entries = f.read().strip().split('\n\n')
entries

['， O\n患 O\n者 O\n3 O\n月 O\n前 O\n因 O\n“ O\n直 B-DSE\n肠 I-DSE\n癌 I-DSE\n” O\n于 O\n在 O\n我 O\n院 O\n于 O\n全 O\n麻 O\n上 O\n行 O\n直 B-OPS\n肠 I-OPS\n癌 I-OPS\n根 I-OPS\n治 I-OPS\n术 I-OPS\n（ I-OPS\nD I-OPS\nI I-OPS\nX I-OPS\nO I-OPS\nN I-OPS\n术 I-OPS\n） I-OPS\n， O\n手 O\n术 O\n过 O\n程 O\n顺 O\n利 O\n， O\n术 O\n后 O\n给 O\n予 O\n抗 O\n感 O\n染 O\n及 O\n营 O\n养 O\n支 O\n持 O\n治 O\n疗 O\n， O\n患 O\n者 O\n恢 O\n复 O\n好 O\n， O\n切 O\n口 O\n愈 O\n合 O\n良 O\n好 O\n。 O\n， O\n术 O\n后 O\n病 O\n理 O\n示 O\n： O\n直 B-DSE\n肠 I-DSE\n腺 I-DSE\n癌 I-DSE\n（ I-DSE\n中 I-DSE\n低 I-DSE\n度 I-DSE\n分 I-DSE\n化 I-DSE\n） I-DSE\n， I-DSE\n浸 I-DSE\n润 I-DSE\n溃 I-DSE\n疡 I-DSE\n型 I-DSE\n， O\n面 O\n积 O\n3 O\n. O\n5 O\n* O\n2 O\nC O\nM O\n， O\n侵 O\n达 O\n外 O\n膜 O\n。 O\n双 O\n端 O\n切 O\n线 O\n另 O\n送 O\n“ O\n近 O\n端 O\n” O\n、 O\n“ O\n远 O\n端 O\n” O\n及 O\n环 O\n周 O\n底 O\n部 O\n切 O\n除 O\n面 O\n未 O\n查 O\n见 O\n癌 O\n。 O\n肠 B-PAT\n壁 I-PAT\n一 I-PAT\n站 I-PAT\n（ I-PAT\n1 I-PAT\n0 I-PAT\n个 I-PAT\n） I-PAT\n、 I-PAT\n中 I-PAT\n间 I-PAT\n组 I-PAT\n（ I-PAT\n8 I-PAT\n个 I-PAT\n） I-PAT\n淋 I-PAT\n巴 I-P

In [37]:
MAX_LEN = 256 - 2
sents, tags_li = [], [] # list of lists
for entry in entries:
    words = [line.split()[0] for line in entry.splitlines()]
    tags = ([line.split()[-1] for line in entry.splitlines()])
#     sents.append(["[CLS]"] + words + ["[SEP]"])  # 每个句子前后加['CLS']和['SEP']
#     tags_li.append(["<PAD>"] + tags + ["<PAD>"])
    if len(words) > MAX_LEN:
        # 先对句号分段
        word, tag = [], []
        for char, t in zip(words, tags):
            
            if char != '。':
                word.append(char)
                tag.append(t)
            else:
#                 if len(word) > MAX_LEN:
#                     word = word[:MAX_LEN]
#                     tag = tag[:MAX_LEN]
                sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
                tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
                word, tag = [], []            
        # 最后的
        if len(word):
            sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
            tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])
            word, tag = [], []
    else:
        sents.append(["[CLS]"] + word[:MAX_LEN] + ["[SEP]"])
        tags_li.append(["PAD"] + tag[:MAX_LEN] + ["PAD"])

In [38]:
len(sents)

6306

In [39]:
len(tags_li)

6306

In [36]:
sents[0], tags_li[0]

(['，',
  '患',
  '者',
  '3',
  '月',
  '前',
  '因',
  '“',
  '直',
  '肠',
  '癌',
  '”',
  '于',
  '在',
  '我',
  '院',
  '于',
  '全',
  '麻',
  '上',
  '行',
  '直',
  '肠',
  '癌',
  '根',
  '治',
  '术',
  '（',
  'D',
  'I',
  'X',
  'O',
  'N',
  '术',
  '）',
  '，',
  '手',
  '术',
  '过',
  '程',
  '顺',
  '利',
  '，',
  '术',
  '后',
  '给',
  '予',
  '抗',
  '感',
  '染',
  '及',
  '营',
  '养',
  '支',
  '持',
  '治',
  '疗',
  '，',
  '患',
  '者',
  '恢',
  '复',
  '好',
  '，',
  '切',
  '口',
  '愈',
  '合',
  '良',
  '好',
  '。',
  '，',
  '术',
  '后',
  '病',
  '理',
  '示',
  '：',
  '直',
  '肠',
  '腺',
  '癌',
  '（',
  '中',
  '低',
  '度',
  '分',
  '化',
  '）',
  '，',
  '浸',
  '润',
  '溃',
  '疡',
  '型',
  '，',
  '面',
  '积',
  '3',
  '.',
  '5',
  '*',
  '2',
  'C',
  'M',
  '，',
  '侵',
  '达',
  '外',
  '膜',
  '。',
  '双',
  '端',
  '切',
  '线',
  '另',
  '送',
  '“',
  '近',
  '端',
  '”',
  '、',
  '“',
  '远',
  '端',
  '”',
  '及',
  '环',
  '周',
  '底',
  '部',
  '切',
  '除',
  '面',
  '未',
  '查',
  '见',
  '癌',
  '。',
  '肠',
  '壁',
  '一',
  '站',